In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

from utils.SineDataSet import get_batch
from utils.Generator import Generator

import glob

In [ ]:
def get_latest_version_path(method_name):
    folder_path = f'trained_models/{method_name}/'
    paths = glob.glob(f'{folder_path}*.pt', recursive=True)
    versions = np.array([int(path[len(folder_path+'G_state_'):-3]) for path in paths])
    return paths[np.argmax(versions)]

n = 50
def moving_average(a, n=n):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
Gs = []
for i, method_name in enumerate(['gtfd', 'supervised', 'stein'], start=1):
    file_path = get_latest_version_path(method_name)
    print(file_path)
    G = Generator().cuda()
    G.load_state_dict(torch.load(file_path))
    G.eval()
    Gs.append(G)

    ticks = np.load(f'trained_models/{method_name}/ticks.npy')
    mean_errors_to_groundtruth = np.load(f'trained_models/{method_name}/mean_errors_to_groundtruth.npy')
    plt.subplot(130+i)
    plt.title(f"Method: {method_name}, min at index {np.argmin(mean_errors_to_groundtruth)} at minute {ticks[np.argmin(mean_errors_to_groundtruth)]//60}")
    plt.plot(ticks, mean_errors_to_groundtruth)
    plt.plot(ticks[n-1:], moving_average(mean_errors_to_groundtruth))
    plt.ylim(bottom=1, top=8)

plt.gcf().set_size_inches(25, 4)
plt.show()
G_gtfd, G_supervised, G_stein = Gs

In [ ]:
def get_psnrs(groundtruth, approximations):
    return np.array([peak_signal_noise_ratio(groundtruth, approximation, data_range=2) for approximation in approximations])

def get_ssims(groundtruth, approximations):
    return np.array([structural_similarity(groundtruth, approximation, data_range=2) for approximation in approximations])

In [ ]:
ys, yδs, _ = get_batch(100)

ys_gtfd = G_gtfd(yδs).detach().cpu().numpy()
ys_supervised = G_supervised(yδs).detach().cpu().numpy()
ys_stein = G_stein(yδs).detach().cpu().numpy()

ys, yδs = ys.detach().cpu().numpy(), yδs.detach().cpu().numpy()

In [ ]:
psnrs = []
ssims = []

for y, yδ, y_gtfd, y_supervised, y_stein in zip(ys, yδs, ys_gtfd, ys_supervised, ys_stein):
    psnrs.append(get_psnrs(y, [yδ, y_gtfd, y_supervised, y_stein]))
    ssims.append(get_ssims(y, [yδ, y_gtfd, y_supervised, y_stein]))

psnrs = np.array(psnrs)
ssims = np.array(ssims)

print('Mean PSNRs:', psnrs.mean(axis=0))
print('Mean SSIMs:', ssims.mean(axis=0))
print("=====")
print('Median PSNRs:', np.median(psnrs, axis=0))
print('Median SSIMs:', np.median(ssims, axis=0))
print("=====")
print('Std. PSNRs:', psnrs.std(axis=0))
print('Std. SSIMs:', ssims.std(axis=0))

In [ ]:
for i, (y, yδ, y_gtfd, y_supervised, y_stein) in enumerate(zip(ys, yδs, ys_gtfd, ys_supervised, ys_stein)):
    plt.subplot(151)
    plt.title('')
    plt.plot(np.linspace(0, 1, 128), y)

    plt.subplot(152)
    plt.title(f'PSNR: {round(peak_signal_noise_ratio(y, yδ, data_range=2), 1)} SSIM: {round(structural_similarity(y, yδ, data_range=2), 2)}')
    plt.plot(np.linspace(0, 1, 128), yδ)

    plt.subplot(153)
    plt.title(f'PSNR: {round(peak_signal_noise_ratio(y, y_gtfd, data_range=2), 1)} SSIM: {round(structural_similarity(y, y_gtfd, data_range=2), 2)}')
    plt.plot(np.linspace(0, 1, 128), y_gtfd)

    plt.subplot(154)
    plt.title(f'PSNR: {round(peak_signal_noise_ratio(y, y_supervised, data_range=2), 1)} SSIM: {round(structural_similarity(y, y_supervised, data_range=2), 2)}')
    plt.plot(np.linspace(0, 1, 128), y_supervised)

    plt.subplot(155)
    plt.title(f'PSNR: {round(peak_signal_noise_ratio(y, y_stein, data_range=2), 1)} SSIM: {round(structural_similarity(y, y_stein, data_range=2), 2)}')
    plt.plot(np.linspace(0, 1, 128), y_stein)

    plt.gcf().set_size_inches(25, 3)
    plt.savefig(f'plots/example{i}.png')
    plt.show()